In [1]:
import pandas as pd

In [6]:
df = pd.read_pickle('/workspaces/lit-gpt/out/inference/pythia-160m-deduped-sharegpt-final-eval.pkl')

In [7]:
df.coherence.mean()

0.943869817353332

In [9]:
df.sort_values(by='coherence').head(10)[['input', 'output', 'coherence']].values

array([['Suppose you face a 2-armed bandit task whose true action values change\nrandomly from time step to time step. Specifically, suppose that, for any time step,\nthe true values of actions 1 and 2 are respectively 10 and 20 with probability 0.5 (case\nA), and 90 and 80 with probability 0.5 (case B). If you are not able to tell which case\nyou face at any step, what is the best expected reward you can achieve and how should\nyou behave to achieve it? Now suppose that on each step you are told whether you are\nfacing case A or case B (although you still don’t know the true action values). This is an\nassociative search task. What is the best expected reward you can achieve in this task,\nand how should you behave to achieve it?',
        'Acknowledged.', 0.000489],
       ['next one is "Head Held High": Devils dress like angels\nPulling off the halos\nwith those bedroom eyes\n\nSee the crooked smiles\nWanna swallow me whole\nto the emptiness inside\n\nBut I keep my head held high\nh

In [4]:
df = pd.read_csv('/workspaces/lit-gpt/out/inference/pythia-160m-deduped-oasst1-base-eval.csv')

In [5]:
df.dropna(subset=['BARTScore']).sort_values(by='BARTScore').tail(5)[['input', 'output', 'BARTScore']].values

array([['What are the five most common causes of stress among people?',
        'how how how how how how how how to use to for building of of the of the of the of the of the of the of the of the of the of the of the of thea of theof the of the of the of theof the of the of the of thea of theof the of theof theof thea of theof the of theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theoftheof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof thetheof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof the

In [29]:
data = pd.read_json('/workspaces/lit-gpt/out/inference/pythia-6.9b-deduped-oasst1-base-eval.json')

In [30]:
scores = data.score.apply(pd.Series).stack().str.split('/').apply(pd.Series)[0].str.strip().replace({'n':'None'}).apply(eval).unstack()#.mean()
# .value_counts()

In [31]:
scores.describe()

,0,1,2,3
count,97.000000,100.000000,28.000000,70.000000
mean,1.927835,1.780000,2.107143,1.614286
std,1.684781,2.002927,2.024650,1.705004
min,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000
75%,2.000000,1.000000,2.000000,1.000000
max,8.000000,10.000000,8.000000,8.000000


In [36]:
col = 3
pd.concat([data, scores], axis=1).dropna(subset=[col]).sort_values(col).head(5)[['user_prompt', 'assistant_prompt', col]].values

array([['Act as the Norse Goddess Freyja.',
        "a is a spirit of love, sexuality, passion, pleasure and fertility. She is a fertility goddess similar to Aphrodite and Venus. She is associated with the moon and with the sea. Her priestesses were known as Freyja's Maids.\n\nThe name Freyja's Maids is a feminine form of the word 'freyja'. This is the name given to the Norse Goddess, Freyja, by the Norse people. A form of Freyja which was popularised by the Icelandic poet, Snorri Sturluson, is known as Freyja's Maid.\n\nFreyja's Maids is a modern name for a Norse goddess. The concept of a female-only group of priestesses who worshipped and served the Goddess Freyja is known as the Freyja's Maids.\n\nFreyja's Maids is a fertility goddess similar to Aphrodite and Venus. Freyja's Maids is associated with the moon and with the sea. Freyja's Maids worshipped and served the Goddess Freyja.\n\nFreyja's Maids worshipped and served the Goddess Freyja. Freyja's Maids offered their love to the G